# Met Office Climate Data Hackathon

## Using xclim with UK Climate Projections data

This notebook shows you how to work with the UK Climate Projections netCDF files available on the CEDA Archive 
using the xarray Python library. It demonstrates:
 * the capability of the xclim Python package

This module requires the following:

*  Python packages in your environment 
 * xarray
 * xclim
 * cartopy
 
* UKCP data files
 * monthly mean surface air temperature (available from CEDA Archive [here](http://dap.ceda.ac.uk/badc/ukcp18/data/land-rcm/uk/12km/rcp85/01/tas/mon/latest/tas_rcp85_land-rcm_uk_12km_01_mon_198012-208011.nc))
 * land mask (available from)
 * daily maximum surface air temperatures (available from CEDA Archive [here](http://data.ceda.ac.uk/badc/ukcp18/data/land-rcm/uk/12km/rcp85/01/tasmax/day/latest))

## Preparatory actions
Load packages

In [1]:
import xclim
import xarray as xr

Assign file locations

In [2]:
UKCP_FILE_DIRECTORY_TASMAX_GCM = '/project/ukcp/land-gcm/uk/60km/rcp26/04/tasmax/day/v20200302'

Load UKCP data to work with

In [3]:
path=UKCP_FILE_DIRECTORY_TASMAX_GCM
fi='/tasmax_rcp26_land-gcm_uk_60km_*.nc'
tasmax_ds=xr.open_mfdataset(path+fi)
tasmax_da=tasmax_ds.tasmax.sel(time=slice('1980','2050'))

AttributeError: 'CFTimeIndex' object has no attribute '_cache'

In [ ]:
print('indices')
dir(xclim.indices)

Calculate some area weights

In [4]:
weights = np.cos(np.deg2rad(tasmax_da.latitude))
weights.name = "weights"
print(weights)

NameError: name 'np' is not defined

In [ ]:
weights.plot.pcolormesh()
plt.show()

Calculate tropical night index with threshold of 25C and present results on annual time-scale

In [ ]:
trop_n=xclim.indices.tropical_nights(tasmax_da,thresh='25 C', freq='AS-DEC')
trop_n_m=trop_n.mean(dim=['projection_y_coordinate','projection_x_coordinate'])  # not area weight

trop_n_w=trop_n.weighted(weights)
trop_n_m_w=trop_n_w.mean(dim=['projection_y_coordinate','projection_x_coordinate'])  # area weighted

Plot both curves

In [ ]:
trop_n_m.plot()
trop_n_m_w.plot()       # not very different as considering a small area 
plt.grid()

Find and plot hottest day of the year

In [ ]:
tx_max=xclim.indices.tx_max(tasmax_da, freq='AS-DEC')
tx_max_m=tx_max.max(dim=['projection_y_coordinate','projection_x_coordinate'])
tx_max_m.plot()
plt.grid()

Why is the final value so low? 

It's because the year has only December in it. We could recalculate with only full years.

In [ ]:
tasmax_da=tasmax_ds.tasmax.sel(time=slice('1979-12','2049-11'))
tx_max=xclim.indices.tx_max(tasmax_da, freq='AS-DEC')
tx_max_m=tx_max.max(dim=['projection_y_coordinate','projection_x_coordinate'])
tx_max_m.plot()
plt.grid()